# Code 24

In [ ]:
import numpy as np

def sigmoid(x):
    '''
    ranges from  0 to 1
    '''
    return 1 / (1 + np.exp(-x))

class Perceptron:
    def __init__(self, num_inputs):
        # initialize the weights randomly, and set the bias to 1
        self.w1 = np.random.random(num_inputs)
        self.b1 = 1
    
    def predict(self, X):
        # compute activation for input layer
        fX = np.dot(X, self.w1) + self.b1
        # non-linear transform
        activation = sigmoid(fX)
        # check threshold: for sigmoid, use 0.5, for tanh, use 0
        y = np.where(fX >= 0.5, 1, -1)
        return y
    
    def fit(self, train_data, train_labels, num_epochs=20):
        models = []
        print(num_epochs)
        for epoch in range(1, num_epochs+1):
            print(epoch)
            for (X, y) in zip(train_data, train_labels):
                pred_label = self.predict(X)

                if pred_label != y:
                    print('update')
                    self.w1 = self.w1 + (X * y)
                    self.b1 = self.b1 + y

            models.append((self.w1, self.b1))

        return models

# Code 25

In [ ]:
and_data = np.array([[1, 1], [1, 0], [0, 1], [0, 0]])
and_labels = np.array([1, -1, -1, -1])

or_data = np.array([[1, 1], [1, 0], [0, 1], [0, 0]])
or_labels = np.array([1, 1, 1, -1])

xor_data = np.array([[1, 1], [1, 0], [0, 1], [0, 0]])
xor_labels = np.array([-1, 1, 1, -1])

# initialize perceptron
perceptron = Perceptron(2)

iters = perceptron.fit(and_data, and_labels, num_epochs=10)
and_predictions = perceptron.predict(and_data)

# Code 26

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense

# input: a sequence  of 2 integers
main_input = Input(shape=(2,), name='main_input')

# add the output layer
output = Dense(2, activation='hard_sigmoid', name='output', kernel_initializer='glorot_uniform')(main_input)

# f(X) = sigmoid(X*W + b)

# the model is specified by connecting input and output
perceptron_keras = Model(inputs=[main_input], outputs=[output])

# compile the model
perceptron_keras.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# train the model on a validation set and dave the loss and accuracy for every epoch
history = perceptron_keras.fit(X, y,
                    epochs=15,
                    verbose=1,
                    validation_split=0.2
                )

# Code 27

In [ ]:
# input: a sequence  of 2 integers
mlp_input = Input(shape=(2,), name='main_input')

# add a hidden layer
mlp_hidden = Dense(16, activation='relu', name='hidden', kernel_initializer='glorot_uniform')(mlp_input)

# add the output layer
mlp_output = Dense(2, activation='softmax', name='output', kernel_initializer='glorot_uniform')(mlp_hidden)

# the model is specified by connecting input and output
mlp = Model(inputs=[mlp_input], outputs=[mlp_output])

mlp.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy']
           )

mlp_history = mlp.fit(X, y,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2)

# Code 28

In [ ]:
dropout = Dropout(0.3, name='dropout')(layer)

# Code 29

In [ ]:
# collect known word tokens and tags
wordset, labelset = set(), set()

# collect all unique labels
labelset.update(set(train_labels))

# collect all word types in the training data
for words in train_instances:
    wordset.update(set(words))

# map words and tags into integer IDs
PAD = '-PAD-'
UNK = '-UNK-'
word2int = {word: i + 2 for i, word in enumerate(sorted(wordset))}
word2int[PAD] = 0  # special token for padding
word2int[UNK] = 1  # special token for unknown words
 
label2int = {label: i for i, label in enumerate(sorted(labelset))}
# structure to translate IDs back to labels
int2label = {i:label for label, i in label2int.items()}

# helper function
def convert2ints(instances):
    result = []
    for words in instances:
        # replace words with ID, use 1 for unknown words
        word_ints = [word2int.get(word, 1) for word in words]
        result.append(word_ints)
    return result

# convert data and labels                          
train_instances_int = convert2ints(train_instances)
train_labels_int = [label2int[label] for label in train_labels]

test_instances_int = convert2ints(test_instances)
test_labels_int = [label2int[label] for label in test_labels]

# convert labels to 1-hot encoding
from keras.utils import to_categorical
train_labels_1hot = to_categorical(train_labels_int, len(label2int))
test_labels_1hot = to_categorical(test_labels_int, len(label2int))

# Code 30

In [ ]:
# compute 95th percentile of training sentence lengths
L = sorted(map(len, train_instances))
MAX_LENGTH = L[int(len(L) * 0.95)]

# apply padding
from keras.preprocessing.sequence import pad_sequences
train_instances_int = pad_sequences(train_instances_int, padding='post', maxlen=MAX_LENGTH)
test_instances_int = pad_sequences(test_instances_int, padding='post', maxlen=MAX_LENGTH)